In [7]:
#CONFIG INICIAL

import requests
import pandas as pd
import os, json, ast
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"accept": "application/json"}
RAW_DIR = "../dados/raw"
os.makedirs(RAW_DIR, exist_ok=True)


def get_json(url, params=None, retries=3):
    for tentativa in range(retries):
        try:
            response = requests.get(url, headers=HEADERS, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"[ERRO] Status {response.status_code} - {url}")
        except Exception as e:
            print(f"[EXCEÇÃO] {e}")
        return None

## 1. Deputados

In [2]:
# 1. Deputados
def get_deputados(legislatura):
    print(f"[INICIANDO] Coleta de deputados da legislatura {legislatura}...")
    url = f"{BASE_URL}/deputados"
    data = get_json(url, {"itens": 1000, "idLegislatura": legislatura})
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, f"deputados_legislatura_{legislatura}.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} deputados salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum dado retornado.")
        return pd.DataFrame()

In [3]:
df_deputados = get_deputados(56)
df_deputados.head()

[INICIANDO] Coleta de deputados da legislatura 56...
[FINALIZADO] 1000 deputados salvos em '../dados/raw\deputados_legislatura_56.csv'


,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PHS,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
1,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PR,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
2,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PL,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
3,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PSC,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,None
4,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None


## 2. Partidos

In [8]:
# 2. Partidos
def get_partidos():
    print("[Início] Partidos")
    data = get_json(f"{BASE_URL}/partidos", params={"itens": 1000, "ordem": "ASC", "ordenarPor": "sigla"})
    df = pd.DataFrame(data["dados"]).rename(columns={
        "id":   "id_partido",
        "sigla":"sigla_partido",
        "nome": "nome_partido"
    })
    df = df.drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, "partidos.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} partidos salvos")
    return df


In [9]:
df_partidos = get_partidos()
df_partidos.head()

[Início] Partidos
[OK] 20 partidos salvos


,id_partido,sigla_partido,nome_partido
0,36898,AVANTE,Avante
1,37905,CIDADANIA,Cidadania
2,36899,MDB,Movimento Democrático Brasileiro
3,37901,NOVO,Partido Novo
4,36779,PCdoB,Partido Comunista do Brasil


## 3. Proposições

In [6]:
# 3. Proposições
def get_proposicoes_por_ano(ano):
    print(f"[Início] Proposições {ano}")
    todas, pag = [], 1
    while True:
        data = get_json(f"{BASE_URL}/proposicoes", {"ano": ano, "pagina": pag, "itens": 1000})
        if not data or not data["dados"]:
            break
        todas.extend(data["dados"]);  pag += 1
    df = pd.DataFrame(todas).rename(columns={
        "id": "id_proposicao",
        "siglaTipo": "sigla_tipo",
        "codTipo": "cod_tipo"
    })
    df = df.drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} proposições salvas")
    return df

In [7]:
for ano in range(2019,2023):
    df_proposicoes = get_proposicoes_por_ano(ano)
df_proposicoes.head()


[Início] Proposições 2019


[OK] 25728 proposições salvas
[Início] Proposições 2020
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes
[OK] 7800 proposições salvas
[Início] Proposições 2021
[OK] 21712 proposições salvas
[Início] Proposições 2022
[OK] 13760 proposições salvas


,id_proposicao,sigla_tipo,cod_tipo,numero,ano,ementa
0,303153,PL,139,618,2022,Dispõe sobre o exercício da profissão de Podól...
1,493361,PL,139,2253,2022,Dispõe sobre o monitoramento por instrumentos ...
2,587207,PL,139,1367,2022,Dispõe sobre a prestação dos serviços de contr...
3,597587,PL,139,3062,2022,"Altera a redação dos arts. 14, 17 e 18 da Lei ..."
4,996806,PL,139,1637,2022,Dispõe sobre a avaliação psicológica de gestan...


AQUI NO MEIO SERIA INTERESSANTE ADICIONAR UM TRATAMENTO NO ARQUIVO proposicoes_2023, POIS A CÉLULA ABAIXO UTILIZA ESSE ARQUIVO COMO INPUT

## 4. Detalhes Proposições

In [8]:
# 4. Detalhes proposições

def get_votacoes_por_proposicoes_parallel(ano, max_workers=10):
    print(f"[Início] Votações → proposições {ano}")
    prop_path = os.path.join(RAW_DIR, f"proposicoes_{ano}.csv")
    props = pd.read_csv(prop_path)["id_proposicao"]
    
    def fetch(prop_id):
        url  = f"{BASE_URL}/proposicoes/{prop_id}/votacoes"
        dat  = get_json(url)
        if dat and "dados" in dat:
            for d in dat["dados"]:
                d["id_proposicao"] = prop_id
            return dat["dados"]
        return []
    
    votacoes = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, pid) for pid in props]
        for i, fut in enumerate(as_completed(futures), 1):
            votacoes.extend(fut.result())
            if i % 500 == 0:
                print(f"{i}/{len(props)} proposições processadas")
    
    df = pd.DataFrame(votacoes)
    if not df.empty:
        df = df.rename(columns={
            "id": "id_votacao",
            "siglaOrgao": "sigla_orgao",
            "dataHoraRegistro": "data_hora_registro"
        }).drop(columns=[c for c in df.columns if "uri" in c.lower()], errors="ignore")
    path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    df.to_csv(path, index=False)
    print(f"[OK] {len(df)} votações salvas")
    return df

for ano in range(2019, 2023):
    get_votacoes_por_proposicoes_parallel(ano)



[Início] Votações → proposições 2019
500/25728 proposições processadas
1000/25728 proposições processadas
1500/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2191911/votacoes
2000/25728 proposições processadas
2500/25728 proposições processadas
3000/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2194537/votacoes
3500/25728 proposições processadas
4000/25728 proposições processadas
4500/25728 proposições processadas
5000/25728 proposições processadas
5500/25728 proposições processadas
6000/25728 proposições processadas
6500/25728 proposições processadas
7000/25728 proposições processadas
7500/25728 proposições processadas
8000/25728 proposições processadas
8500/25728 proposições processadas
9000/25728 proposições processadas
9500/25728 proposições processadas
10000/25728 proposições processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/proposicoes/2204096

In [10]:
#for ano in range(2019, 2023):
#    get_votacoes_por_proposicoes_parallel(ano)
#df_votacoes.head()

## 5. Votos individuais

In [2]:
# 5. Votos individuais referentes as proposições
def get_votos_individuais_proposicoes_raw(ano, max_workers=10):
    print(f"[Início] Coleta de votos individuais brutos - {ano}")
    
    vot_path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    votacoes = pd.read_csv(vot_path)["id_votacao"].dropna().tolist()

    def fetch(vot_id):
        data = get_json(f"{BASE_URL}/votacoes/{vot_id}/votos")
        if data and "dados" in data:
            for d in data["dados"]:
                d["id_votacao"] = vot_id
            return data["dados"]
        return []

    votos = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, vid) for vid in votacoes]
        for i, fut in enumerate(as_completed(futures), 1):
            try:
                votos.extend(fut.result())
            except Exception as e:
                print(f"[ERRO] Erro ao processar votação: {e}")
            if i % 250 == 0:
                print(f"{i}/{len(votacoes)} votações processadas")

    # Converte para DataFrame e salva sem nenhuma modificação
    df = pd.DataFrame(votos)
    path_out = os.path.join(RAW_DIR, f"votos_individuais_proposicoes_raw_{ano}.csv")
    df.to_csv(path_out, index=False)
    print(f"[OK] {len(df)} votos salvos (dados brutos)")
    return df



In [4]:

for ano in range(2019, 2023):
    get_votos_individuais_proposicoes_raw(ano)
#df_votos.head()

[Início] Coleta de votos individuais brutos - 2019
250/11450 votações processadas
500/11450 votações processadas
750/11450 votações processadas
1000/11450 votações processadas
1250/11450 votações processadas
1500/11450 votações processadas
1750/11450 votações processadas
2000/11450 votações processadas
2250/11450 votações processadas
2500/11450 votações processadas
2750/11450 votações processadas
3000/11450 votações processadas
3250/11450 votações processadas
3500/11450 votações processadas
3750/11450 votações processadas
4000/11450 votações processadas
4250/11450 votações processadas
4500/11450 votações processadas
4750/11450 votações processadas
5000/11450 votações processadas
5250/11450 votações processadas
5500/11450 votações processadas
5750/11450 votações processadas
6000/11450 votações processadas
6250/11450 votações processadas
6500/11450 votações processadas
6750/11450 votações processadas
7000/11450 votações processadas
7250/11450 votações processadas
7500/11450 votações proc

## 6. Orientações partidárias

In [6]:
def get_orientacoes_por_proposicoes_raw(ano, max_workers=10):
    print(f"[Início] Coleta bruta - orientações partidárias {ano}")
    
    vot_path = os.path.join(RAW_DIR, f"votacoes_proposicoes_{ano}.csv")
    votacoes = pd.read_csv(vot_path)["id_votacao"].dropna().tolist()

    def fetch(vot_id):
        data = get_json(f"{BASE_URL}/votacoes/{vot_id}/orientacoes")
        if data and "dados" in data:
            for d in data["dados"]:
                d["id_votacao"] = vot_id
            return data["dados"]
        return []

    orientacoes = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(fetch, vid) for vid in votacoes]
        for i, fut in enumerate(as_completed(futures), 1):
            try:
                orientacoes.extend(fut.result())
            except Exception as e:
                print(f"[ERRO] Erro ao processar votação: {e}")
            if i % 250 == 0:
                print(f"{i}/{len(votacoes)} votações processadas")

    df = pd.DataFrame(orientacoes)
    path_out = os.path.join(RAW_DIR, f"orientacoes_proposicoes_raw_{ano}.csv")
    df.to_csv(path_out, index=False)
    print(f"[OK] {len(df)} orientações salvas (dados brutos)")
    return df


In [7]:

for ano in range(2019, 2023):
    get_orientacoes_por_proposicoes(ano)

#df_orientacoes.head()


[Início] Orientações partidárias 2019
250/11450 votações processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/1672576-97/orientacoes
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/2149372-22/orientacoes
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/2080604-393/orientacoes
500/11450 votações processadas
750/11450 votações processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/2190355-51/orientacoes
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/2190585-101/orientacoes
1000/11450 votações processadas
1250/11450 votações processadas
1500/11450 votações processadas
[ERRO] Status 504 - https://dadosabertos.camara.leg.br/api/v2/votacoes/2192459-704/orientacoes
1750/11450 votações processadas
2000/11450 votações processadas
2250/11450 votações processadas
2500/11450 votações processadas
2750/11450 votações processadas
3000/11450 votações processadas
3250/11450 vot

## 7. Autores/temas das proposições

In [2]:
def fetch_autores(prop_id, retries=3):
    for tentativa in range(retries):
        d = get_json(f"{BASE_URL}/proposicoes/{prop_id}/autores")
        if d and "dados" in d:
            return [{"id_proposicao": prop_id, **a} for a in d["dados"]]
        time.sleep(1)
    print(f"[ERRO] Falha autores → id_proposicao {prop_id}")
    return []

def fetch_temas(prop_id, retries=3):
    for tentativa in range(retries):
        d = get_json(f"{BASE_URL}/proposicoes/{prop_id}/temas")
        if d and "dados" in d:
            return [{"id_proposicao": prop_id, **t} for t in d["dados"]]
        time.sleep(1)
    print(f"[ERRO] Falha temas → id_proposicao {prop_id}")
    return []

def coleta_autores_temas(ano, max_workers=10):
    print(f"[Início] Autores & Temas {ano}")
    props = pd.read_csv(os.path.join(RAW_DIR, f"proposicoes_{ano}.csv"))["id_proposicao"]
    
    autores, temas = [], []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs_autores = {ex.submit(fetch_autores, pid): pid for pid in props}
        futs_temas = {ex.submit(fetch_temas, pid): pid for pid in props}

        for i, fut in enumerate(as_completed(futs_autores), 1):
            autores.extend(fut.result())
            if i % 500 == 0:
                print(f"[Autores] {i}/{len(futs_autores)} proposições")

        for i, fut in enumerate(as_completed(futs_temas), 1):
            temas.extend(fut.result())
            if i % 500 == 0:
                print(f"[Temas] {i}/{len(futs_temas)} proposições")

    pd.DataFrame(autores).to_csv(os.path.join(RAW_DIR, f"autores_proposicoes_{ano}.csv"), index=False)
    pd.DataFrame(temas).to_csv(os.path.join(RAW_DIR, f"temas_proposicoes_{ano}.csv"), index=False)
    print(f"[OK] Autores ({len(autores)}) & Temas ({len(temas)}) salvos")


In [3]:
for ano in range(2019, 2023):
    coleta_autores_temas(ano)
#df_autores.head()

[Início] Autores & Temas 2019
[Autores] 500/25728 proposições
[Autores] 1000/25728 proposições
[Autores] 1500/25728 proposições
[Autores] 2000/25728 proposições
[Autores] 2500/25728 proposições
[Autores] 3000/25728 proposições
[Autores] 3500/25728 proposições
[Autores] 4000/25728 proposições
[Autores] 4500/25728 proposições
[Autores] 5000/25728 proposições
[Autores] 5500/25728 proposições
[Autores] 6000/25728 proposições
[Autores] 6500/25728 proposições
[Autores] 7000/25728 proposições
[Autores] 7500/25728 proposições
[Autores] 8000/25728 proposições
[Autores] 8500/25728 proposições
[Autores] 9000/25728 proposições
[Autores] 9500/25728 proposições
[Autores] 10000/25728 proposições
[Autores] 10500/25728 proposições
[Autores] 11000/25728 proposições
[Autores] 11500/25728 proposições
[Autores] 12000/25728 proposições
[Autores] 12500/25728 proposições
[Autores] 13000/25728 proposições
[Autores] 13500/25728 proposições
[Autores] 14000/25728 proposições
[Autores] 14500/25728 proposições
[Aut